In [10]:
from scipy.sparse import linalg as sp_linalg
from timeit import default_timer as timer
from utilities import *
            
## definitions
# honeycomb and reciprocal lattice vecs
a = 1
u1 = np.array((a * np.sqrt(3), 0))
u2 = np.array((a * np.sqrt(3) / 2, a * 3 / 2))
q1, q2 = get_inv(u1, u2)
v1 = np.array((a * np.sqrt(3) / 2, a / 2))
v2 = np.array((-a * np.sqrt(3) / 2, a / 2))
v3 = np.array((0, a))

# moire lattice vecs
lm1, lm2 = 26, 25
u_moire_1, u_moire_2 = lm1 * u1 + lm2 * u2, (lm1 + lm2) * u2 - lm2 * u1
moire_cell = Cell(u_moire_1, u_moire_2)

# twisting angle
theta_twist = lm1 ** 2 + lm1 * lm2 / 2
theta_twist /= np.sqrt(lm1 ** 2 + lm2 ** 2 + lm1 * lm2)
theta_twist /= lm1
theta_twist = np.arccos(theta_twist)
theta_twist = 2 * theta_twist - np.pi / 3
theta_deg = np.rad2deg(theta_twist)
print(theta_deg)

# big parallelogram which contains moire unit cell
l_big_1, l_big_2 = (lm1 + lm2) * 2, (lm1 + 2 * lm2) * 2
u_big_1, u_big_2 = l_big_1 * u1, l_big_2 * u2
big_cell = Cell(u_big_1, u_big_2)

# hoppings
t0 = -2.7
t_inter = t0 * 0.2

-1.2971890475933494


In [13]:
lattice_0, lattice_1 = Lattice(moire_cell), Lattice(moire_cell)
for i1 in range(l_big_1 + 1):
    for i2 in range(l_big_2 + 1):
        vecs = [i1 * u1 + i2 * u2 + big_cell.u_origin, i1 * u1 + i2 * u2 + v3 + big_cell.u_origin]
        for vec in vecs:
            lattice_0.add_site(vec)
        vecs = [rotate_vec(vec, theta_twist) for vec in vecs]
        for vec in vecs:
            lattice_1.add_site(vec)

hamdic_00 = Hamdic((lattice_0.num_sites, lattice_0.num_sites))
vecs = [v1, v2, v3, -v1, -v2, -v3]
for vec in vecs:
    hamdic_00 += lattice_0.get_hamdic_intra(vec, t0)
hamdic_11 = Hamdic((lattice_1.num_sites, lattice_1.num_sites))
vecs = [rotate_vec(vec, theta_twist) for vec in vecs]
for vec in vecs:
    hamdic_11 += lattice_1.get_hamdic_intra(vec, t0)
hamdic_01 = Hamdic((lattice_0.num_sites, lattice_1.num_sites))
hamdic_01 += lattice_0.get_hamdic_inter(lattice_1, t_inter, distance_cutoff_sq=a**2)
hamdic_10 = hamdic_01.get_hermitian_conjugate()

#hamdic = compose_hamdics([[hamdic_00, hamdic_01], [hamdic_10, hamdic_11]])
hamdic = compose_hamdics([[None, None], [None, hamdic_11]])
for key in hamdic.dic.keys():
    #hamdic.dic[key] = hamdic.dic[key].tocsr()
    hamdic.dic[key] = hamdic.dic[key].todense()
    
q_test = np.array([0.123, 0.123])
ham = np.zeros((nd, nd), dtype=np.complex128)
for key in hamdic.dic.keys():
    u_shift = moire_cell.get_u_shift(key[0], key[1])
    ham += hamdic.dic[key] * np.exp(1j * 2 * np.pi * q_test.dot(u_shift))
non_hermicity = np.linalg.norm(ham - ham.T.conjugate())
print("|H - H^dag| = {0:.2f}".format(non_hermicity))

In [ ]:
## calculating the eigenergies along the 1D line
nq, nd = 51, lattice_0.num_sites + lattice_1.num_sites
#nq, nd = 51, lattice_0.num_sites
ns = 10
#ns = nd
q_arr = [moire_cell.q1 * iq / nq for iq in range(nq)]
q_arr += [moire_cell.q1 + (moire_cell.q2 - moire_cell.q1) * 0.5 * iq / nq for iq in range(nq)]
q_arr += [(moire_cell.q2 + moire_cell.q1) * 0.5 * (1 - iq / nq) for iq in range(nq)]

#es_arr = np.zeros((len(q_arr), nd))
es_arr = np.zeros((len(q_arr), ns))
for iq in range(len(q_arr)):
    print("iq: {}   ".format(iq), end="\r")
    q_num = q_arr[iq]
    ham = np.zeros((nd, nd), dtype=np.complex128)
    for key in hamdic.dic.keys():
        u_shift = moire_cell.get_u_shift(key[0], key[1])
        ham += hamdic.dic[key] * np.exp(1j * 2 * np.pi * q_num.dot(u_shift))
    non_hermicity = np.linalg.norm(ham - ham.T.conjugate())
    if non_hermicity > 1e-10:
        print("Error: |H - H^dag| = {0:.2f}".format(non_hermicity))
        raise(Exception)
    #es, vs = np.linalg.eigh(ham)
    es, vs = sp_linalg.eigsh(ham, k=ns, sigma=0, tol=1e-4)
    es_arr[iq,:] = es[:]

In [9]:
%matplotlib qt5
fig = plt.figure()

fax = fig.add_axes((.15, .15, .75, .75))
for jd in range(ns):
    cax = fax.plot(range(len(q_arr)), es_arr[:,jd], c="black", lw=0, marker='.', ms=2.5)
cax = fax.plot([0, len(q_arr)], [0, 0], c="red", linestyle=':')
cax = fax.scatter([0, nq, 2*nq, 3*nq], [0, 0, 0, 0], marker='|', c="red")
    
plt.savefig("figs/ens_1d.lm1_{}.lm2_{}.theta_{:.2f}.t0_{:.2f}.tInter_{:.2f}.pdf".format(lm1, lm2, theta_deg, t0, t_inter))
plt.show()

In [5]:
## calculating the eigenergies in 2D
qx1, qx2 = moire_cell.q2[0], moire_cell.q1[0]
qy1, qy2 = min(0, moire_cell.q1[1]), max(moire_cell.q2[1], (moire_cell.q1+moire_cell.q2)[1])

nqx, nqy, nd = 51, 51, lattice_0.num_sites + lattice_1.num_sites
#nqx, nqy, nd = 51, 51, lattice_0.num_sites
qx_arr = np.linspace(qx1-0.03, qx2+0.03, nqx)
qy_arr = np.linspace(qy1-0.03, qy2+0.03, nqy)
qx_np, qy_np = np.meshgrid(qx_arr, qy_arr)

## diagonalizing
es_np = np.zeros((nqy, nqx, nd))
start = timer()
for iqy in range(nqy):
    print("iqy: {}   ".format(iqy), end="\r")
    for iqx in range(nqx):
        qx, qy = qx_arr[iqx], qy_arr[iqy]
        q_num = np.array((qx, qy))
        ham = np.zeros((lattice_1.num_sites, lattice_1.num_sites), dtype=np.complex128)
        for key in hamdic.dic.keys():
            u_shift = lattice_1.cell.get_u_shift(key[0], key[1])
            ham += hamdic.dic[key] * np.exp(1j * 2 * np.pi * q_num.dot(u_shift))
        non_hermicity = np.linalg.norm(ham - ham.T.conjugate())
        if non_hermicity > 1e-10:
            print("Warning: |H - H^dag| = {0:.2f}".format(non_hermicity))
            raise(Exception)
        es, vs = np.linalg.eigh(ham)
        es_np[iqy,iqx,:] = es[:]
print("Calculating the spectrum took {0}".format(timer()-start))
        
## sorting
es_np_s = np.zeros((nqy, nqx, nd))
for iqy in range(nqy):
    for iqx in range(nqx):
        indices = np.argsort(es_np[iqy,iqx,:])
        es_np_s[iqy,iqx,:] = es_np[iqy,iqx,indices]

Calculating the spectrum took 2.162268399999995


In [24]:
%matplotlib qt5
fig = plt.figure()

fax = fig.add_axes((.15, .15, .75, .75))
cax = fax.pcolormesh(qx_np, qy_np, es_np_s[:,:,38], cmap='viridis')
cbar = plt.colorbar(cax)
moire_cell.draw_q(fax)

#plt.savefig("figs/ens.2d.lm1_{}.lm2_{}.t0_{:.2f}.tInter_{:.2f}.pdf".format(lm1, lm2, t0, tInter))
plt.show()

In [15]:
## plotting the lattice
%matplotlib qt5
fig = plt.figure()

fax = fig.add_axes((.15, .15, .75, .75))
moire_cell.draw(fax)
lattice_0.draw(fax, color_int="red", color_ext="gray", u_offset=np.array([0.01, 0.01]), draw_numbers=False)
#lattice_1.draw(fax, color_int="blue", color_ext="gray", u_offset=np.array([-0.01, 0.01]), draw_numbers=True)
big_cell.draw(fax)
#big_cell_1 = big_cell.rotate(theta_twist)
#big_cell_1.draw(fax)

#plt.savefig("figs/lattice.lm1_{}.lm2_{}.pdf".format(lm1, lm2))
plt.show()